In [19]:
import ast
import pandas as pd
import os
import json

from mappymatch import package_root
from mappymatch.constructs.trace import Trace
from mappymatch.utils.plot import plot_trace
from mappymatch.constructs.geofence import Geofence
from mappymatch.utils.plot import plot_geofence

In [2]:
def read_trip_data(csv_file):
    # Read the CSV file
    df = pd.read_csv(csv_file)

    # Convert string representations of lists back to lists
    df['trajectory'] = df['trajectory'].apply(ast.literal_eval)
    df['velocity_profile'] = df['velocity_profile'].apply(ast.literal_eval)
    df['altitude_profile'] = df['altitude_profile'].apply(ast.literal_eval)

    return df

In [3]:
trip_data_read = read_trip_data('data/trips/Murphy/TL5-218_2020W33_trip_data.csv')
print(trip_data_read.head())

           trip_start_time            trip_end_time  travel_time  \
0  2020-08-10 05:59:15.000  2020-08-10 06:01:59.000        164.0   
1  2020-08-10 06:02:00.000  2020-08-10 06:02:14.000         14.0   
2  2020-08-10 06:28:47.100  2020-08-10 08:01:20.100       5553.0   
3  2020-08-10 08:11:58.900  2020-08-10 08:44:03.900       1925.0   
4  2020-08-10 09:42:13.400  2020-08-10 10:32:56.400       3043.0   

                                    altitude_profile  \
0  [531.9, 531.9, 531.9, 531.9, 531.9, 531.9, 531...   
1  [276.0, 276.0, 276.0, 275.8, 275.7, 275.6, 275...   
2  [294.5, 293.7, 292.7, 291.9, 291.3, 290.6, 290...   
3  [275.5, 274.2, 273.3, 272.8, 272.4, 272.1, 271...   
4  [323.9, 323.5, 323.1, 322.7, 322.2, 321.7, 321...   

                                    velocity_profile   weight  total_fuel  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   8000.0    0.203792   
1  [3.4102, 3.4648, 3.4258, 3.4805, 3.4922, 3.507...   8000.0    0.016903   
2  [3.8867, 5.1758, 6.2

In [4]:
first_trip_trajectory = trip_data_read['trajectory'].iloc[3]

# Convert the list of tuples (latitude, longitude) into a DataFrame
trajectory_df = pd.DataFrame(first_trip_trajectory, columns=['latitude', 'longitude'])

print(trajectory_df.head())

    latitude  longitude
0  44.894679 -91.873756
1  44.894680 -91.873760
2  44.894682 -91.873763
3  44.894684 -91.873766
4  44.894685 -91.873768


In [5]:
trace = Trace.from_dataframe(trajectory_df, lat_column="latitude", lon_column="longitude")

In [9]:
def find_bounding_box_for_all_trips(directory_path):
    # Initialize min and max values for latitude and longitude
    min_lat, max_lat = float('inf'), float('-inf')
    min_lon, max_lon = float('inf'), float('-inf')

    # Iterate through each CSV file in the directory
    for file_name in os.listdir(directory_path):
        if file_name.endswith('_trip_data.csv'):  # Check if the file is a trip data CSV
            file_path = os.path.join(directory_path, file_name)
            trip_data = read_trip_data(file_path)
            
            # Iterate through each trip's trajectory
            for trajectory in trip_data['trajectory']:
                # Extract latitudes and longitudes from the trajectory
                latitudes, longitudes = zip(*trajectory)
                
                # Update min and max values
                min_lat, max_lat = min(min_lat, min(latitudes)), max(max_lat, max(latitudes))
                min_lon, max_lon = min(min_lon, min(longitudes)), max(max_lon, max(longitudes))

    # The bounding box is defined by the minimum and maximum latitude and longitude
    bounding_box = {
        'min_latitude': min_lat,
        'max_latitude': max_lat,
        'min_longitude': min_lon,
        'max_longitude': max_lon
    }

    return bounding_box

In [12]:
def bounding_box_to_geojson(bounding_box, output_file):
    # Define the coordinates of the bounding box (Polygon)
    # The coordinates list must start and end at the same point, forming a closed loop
    coordinates = [
        [
            [bounding_box['min_longitude'], bounding_box['min_latitude']],  # Lower-left corner
            [bounding_box['min_longitude'], bounding_box['max_latitude']],  # Upper-left corner
            [bounding_box['max_longitude'], bounding_box['max_latitude']],  # Upper-right corner
            [bounding_box['max_longitude'], bounding_box['min_latitude']],  # Lower-right corner
            [bounding_box['min_longitude'], bounding_box['min_latitude']]   # Closing the loop at lower-left corner
        ]
    ]
    
    # Define the GeoJSON structure
    geojson_object = {
        "type": "Feature",
        "properties": {},  # Properties can be added if needed
        "geometry": {
            "type": "Polygon",
            "coordinates": coordinates
        }
    }
    
    # Write the GeoJSON object to a file
    with open(output_file, 'w') as f:
        json.dump(geojson_object, f, indent=4)
    
    print(f"GeoJSON file saved to {output_file}")

In [ ]:
murphy_folder = 'data/trips/Murphy'
bounding_box = find_bounding_box_for_all_trips(murphy_folder)
print(bounding_box)

In [13]:
output_geojson_file = 'results/bounding_box.geojson'
bounding_box_to_geojson(bounding_box, output_geojson_file)

GeoJSON file saved to results/bounding_box.geojson


In [16]:
geofence = Geofence.from_geojson(output_geojson_file)

In [17]:
geofence

In [ ]:
plot_trace(trace, point_color="black", m=plot_geofence(geofence))